In [245]:
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
import re

historical_utilisation_df = pd.read_csv("../historical_data/historical_monthly_resource_utilisation.csv")

historical_utilisation_df

,month,n_CC70,n_CC71,n_CC72,n_H70,n_H71,total_time_CC70,total_time_CC71,total_time_CC72,total_time_H70,total_time_H71
0,2022-08-01,28.0,9.0,20.0,85.0,32.0,2580.0,740.0,1767.0,7715.0,3205.0
1,2022-09-01,22.0,1.0,8.0,73.0,35.0,1599.0,113.0,653.0,6704.0,3356.0
2,2022-10-01,31.0,6.0,37.0,65.0,38.0,2700.0,531.0,2741.0,5804.0,3779.0
3,2022-11-01,38.0,5.0,21.0,60.0,31.0,2843.0,414.0,1676.0,4970.0,2910.0
4,2022-12-01,41.0,4.0,24.0,67.0,28.0,4009.0,763.0,1691.0,7024.0,3138.0
5,2023-01-01,27.0,4.0,27.0,68.0,29.0,2781.0,345.0,1838.0,6321.0,2726.0
6,2023-02-01,32.0,12.0,44.0,66.0,13.0,2095.0,1025.0,3111.0,5769.0,1082.0
7,2023-03-01,50.0,20.0,28.0,65.0,NaN,4360.0,1590.0,2574.0,5573.0,NaN
8,2023-04-01,24.0,16.0,38.0,89.0,15.0,1856.0,1149.0,2755.0,7454.0,1302.0
9,2023-05-01,15.0,16.0,35.0,103.0,19.0,1357.0,1307.0,3141.0,9117.0,2110.0


In [246]:
def calculate_theoretical_time(historical_data, rota_data, service_data, long_format_df=True):
    """
    Note that this function has been partially provided by ChatGPT.
    """

    # Convert data into DataFrames
    historical_df = pd.DataFrame(historical_data)
    rota_df = pd.DataFrame(rota_data)
    service_df = pd.DataFrame(service_data)

    # Convert date columns to datetime format
    historical_df['month'] = pd.to_datetime(historical_df['month'])
    service_df['service_start_date'] = pd.to_datetime(service_df['service_start_date'])
    service_df['service_end_date'] = pd.to_datetime(service_df['service_end_date'])

    # Initialize dictionary to store results
    theoretical_availability = {}

    # Iterate over each row in the historical dataset
    for index, row in historical_df.iterrows():
        month_start = row['month']
        month_end = month_start + pd.offsets.MonthEnd(0)
        days_in_month = (month_end - month_start).days + 1

        # Store theoretical available time for each resource
        month_data = {}

        for _, rota in rota_df.iterrows():
            callsign = rota['callsign']

            # Determine summer or winter schedule
            is_summer = month_start.month in range(3, 11)
            start_hour = rota['summer_start'] if is_summer else rota['winter_start']
            end_hour = rota['summer_end'] if is_summer else rota['winter_end']

            # Handle cases where the shift ends after midnight
            if end_hour < start_hour:
                daily_available_time = (24 - start_hour) + end_hour
            else:
                daily_available_time = end_hour - start_hour

            total_available_time = daily_available_time * days_in_month * 60  # Convert to minutes

            # Adjust for servicing periods
            service_downtime = 0
            for _, service in service_df[service_df['resource'] == callsign].iterrows():
                service_start = max(service['service_start_date'], month_start)
                service_end = min(service['service_end_date'], month_end)

                if service_start <= service_end:  # Overlapping service period
                    service_days = (service_end - service_start).days + 1
                    service_downtime += service_days * daily_available_time * 60

            # Final available time after accounting for servicing
            month_data[callsign] = total_available_time - service_downtime

        theoretical_availability[month_start.strftime('%Y-%m-01')] = month_data

    theoretical_availability_df = pd.DataFrame(theoretical_availability).T
    theoretical_availability_df.index.name = "month"
    theoretical_availability_df = theoretical_availability_df.reset_index()
    # theoretical_availability_df = theoretical_availability_df.add_prefix("theoretical_availability_")

    theoretical_availability_df.fillna(0.0)

    if long_format_df:
        theoretical_availability_df = (
            theoretical_availability_df
            .melt(id_vars="month")
            .rename(columns={"value":"theoretical_availability", "variable": "callsign"})
            )

        theoretical_availability_df['theoretical_availability'] = theoretical_availability_df['theoretical_availability'].astype('float')

    return theoretical_availability_df


In [247]:
theoretical_availability_df = calculate_theoretical_time(historical_data=historical_utilisation_df,
                                    rota_data=pd.read_csv("../actual_data/HEMS_ROTA.csv"),
                                    service_data=pd.read_csv("../data/service_dates.csv"),
                                    long_format_df= True
)

theoretical_availability_df

,month,callsign,theoretical_availability
0,2022-08-01,H70,35340.0
1,2022-09-01,H70,34200.0
2,2022-10-01,H70,35340.0
3,2022-11-01,H70,29640.0
4,2022-12-01,H70,14820.0
...,...,...,...
140,2024-08-01,CC72,18600.0
141,2024-09-01,CC72,18000.0
142,2024-10-01,CC72,18600.0
143,2024-11-01,CC72,18000.0


In [248]:
historical_utilisation_df_times = historical_utilisation_df.set_index('month').filter(like='total_time').reset_index()
historical_utilisation_df_times.columns = [x.replace("total_time_","") for x in historical_utilisation_df_times.columns]
historical_utilisation_df_times = historical_utilisation_df_times.melt(id_vars="month").rename(columns={"value":"usage", "variable": "callsign"})
historical_utilisation_df_times = historical_utilisation_df_times.fillna(0)
historical_utilisation_df_times

,month,callsign,usage
0,2022-08-01,CC70,2580.0
1,2022-09-01,CC70,1599.0
2,2022-10-01,CC70,2700.0
3,2022-11-01,CC70,2843.0
4,2022-12-01,CC70,4009.0
...,...,...,...
140,2024-08-01,H71,3158.0
141,2024-09-01,H71,118.0
142,2024-10-01,H71,0.0
143,2024-11-01,H71,1089.0


In [249]:
historical_utilisation_df_complete = pd.merge(
    left=historical_utilisation_df_times,
    right=theoretical_availability_df,
    on=["callsign", "month"],
    how="left"
)

historical_utilisation_df_complete["percentage_utilisation"] = historical_utilisation_df_complete["usage"] / historical_utilisation_df_complete["theoretical_availability"]

historical_utilisation_df_complete["percentage_utilisation_display"] = historical_utilisation_df_complete["percentage_utilisation"].apply(lambda x: f"{x:.1%}")
historical_utilisation_df_complete

,month,callsign,usage,theoretical_availability,percentage_utilisation,percentage_utilisation_display
0,2022-08-01,CC70,2580.0,35340.0,0.073005,7.3%
1,2022-09-01,CC70,1599.0,34200.0,0.046754,4.7%
2,2022-10-01,CC70,2700.0,35340.0,0.076401,7.6%
3,2022-11-01,CC70,2843.0,34200.0,0.083129,8.3%
4,2022-12-01,CC70,4009.0,35340.0,0.113441,11.3%
...,...,...,...,...,...,...
140,2024-08-01,H71,3158.0,22320.0,0.141487,14.1%
141,2024-09-01,H71,118.0,21600.0,0.005463,0.5%
142,2024-10-01,H71,0.0,22320.0,0.000000,0.0%
143,2024-11-01,H71,1089.0,4800.0,0.226875,22.7%


In [250]:
px.bar(historical_utilisation_df_complete, x="month", y="percentage_utilisation", facet_col="callsign")

In [251]:
px.box(historical_utilisation_df_complete, x="percentage_utilisation", y="callsign")

In [252]:
(historical_utilisation_df_complete.groupby('callsign')['percentage_utilisation'].agg(['min', 'max', 'mean', 'median'])*100).round(2)

,min,max,mean,median
callsign,,,,
CC70,2.45,16.96,8.29,7.30
CC71,0.00,15.20,3.71,3.14
CC72,2.35,20.87,10.76,9.50
H70,7.73,47.40,22.05,21.58
H71,0.00,73.81,16.96,14.02


In [253]:
def get_params_df():
    return pd.read_csv("../data/run_params_used.csv")
params_df = get_params_df()

In [254]:
import _processing_functions

In [255]:
from _vehicle_calculation import calculate_available_hours

In [256]:
daily_available_hours, total_avail_hours, total_avail_minutes = calculate_available_hours(params_df)
daily_available_hours

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:90: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-28,False,11,11,2,2,2
1,2025-03-01,False,19,19,10,10,10
2,2025-03-02,False,19,19,10,10,10
3,2025-03-03,False,19,19,10,10,10
4,2025-03-04,False,19,19,10,10,10
...,...,...,...,...,...,...,...
361,2026-02-24,False,19,19,10,10,10
362,2026-02-25,False,19,19,10,10,10
363,2026-02-26,False,19,19,10,10,10
364,2026-02-27,False,19,19,10,10,10


In [257]:
total_avail_hours

,callsign,total_available_hours_in_sim,callsign_group
0,H70,6935,70
1,C70,6935,70
2,H71,4016,71
3,C71,4016,71
4,C72,3650,72


In [258]:
total_avail_minutes

,callsign,total_available_minutes_in_sim,callsign_group
0,H70,416100,70
1,C70,416100,70
2,H71,240960,71
3,C71,240960,71
4,C72,219000,72


In [259]:
import pandas as pd
from datetime import datetime, timedelta

def calculate_theoretical_time_v2(params_df,
rota_data=pd.read_csv("../actual_data/HEMS_ROTA.csv"),
service_data=pd.read_csv("../data/service_dates.csv"),
output_by_month=False,
long_format_df=False
):
    # Convert data into DataFrames
    warm_up_end = _processing_functions.get_param("warm_up_end_date", params_df)
    warm_up_end = datetime.strptime(warm_up_end, "%Y-%m-%d %H:%M:%S")

    sim_end = _processing_functions.get_param("sim_end_date", params_df)
    sim_end = datetime.strptime(sim_end, "%Y-%m-%d %H:%M:%S")

#     hems_rota = pd.read_csv(rota_path)
    # service_dates = pd.read_csv(service_data_path)

    date_range = pd.date_range(start=warm_up_end.date(),
                            end=sim_end.date(),
                            freq='D')
    daily_df = pd.DataFrame({'date': date_range})

#     daily_df = pd.DataFrame(daily_data)
    rota_df = pd.DataFrame(rota_data)
    service_df = pd.DataFrame(service_data)

    # Convert date columns to datetime format
    daily_df['date'] = pd.to_datetime(daily_df['date'])
    service_df['service_start_date'] = pd.to_datetime(service_df['service_start_date'])
    service_df['service_end_date'] = pd.to_datetime(service_df['service_end_date'])

    # Initialize dictionary to store results
    theoretical_availability = {}

    # Iterate over each row in the daily dataset
    for index, row in daily_df.iterrows():
        current_date = row['date']

        # Store theoretical available time for each resource
        day_data = {}

        for _, rota in rota_df.iterrows():
            callsign = rota['callsign']

            # Determine summer or winter schedule
            is_summer = current_date.month in range(3, 11)
            start_hour = rota['summer_start'] if is_summer else rota['winter_start']
            end_hour = rota['summer_end'] if is_summer else rota['winter_end']

            # Handle cases where the shift ends after midnight
            if end_hour < start_hour:
                daily_available_time = (24 - start_hour) + end_hour
            else:
                daily_available_time = end_hour - start_hour

            total_available_time = daily_available_time * 60  # Convert to minutes

            # Adjust for servicing periods
            service_downtime = 0
            for _, service in service_df[service_df['resource'] == callsign].iterrows():
                if service['service_start_date'] <= current_date <= service['service_end_date']:
                    service_downtime = daily_available_time * 60
                    break

            # Final available time after accounting for servicing
            day_data[callsign] = total_available_time - service_downtime

        theoretical_availability[current_date.strftime('%Y-%m-%d')] = day_data

    theoretical_availability_df = pd.DataFrame(theoretical_availability).T
    theoretical_availability_df.index.name = "month"
    theoretical_availability_df = theoretical_availability_df.reset_index()
    # theoretical_availability_df = theoretical_availability_df.add_prefix("theoretical_availability_")

    theoretical_availability_df.fillna(0.0)

    if long_format_df:
        theoretical_availability_df = (
            theoretical_availability_df
            .melt(id_vars="month")
            .rename(columns={"value":"theoretical_availability", "variable": "callsign"})
            )

        theoretical_availability_df['theoretical_availability'] = theoretical_availability_df['theoretical_availability'].astype('float')

    daily_available_minutes = theoretical_availability_df.copy()

    total_avail_minutes = daily_available_minutes.groupby('callsign')[['theoretical_availability']].sum(numeric_only=True).reset_index().rename(columns={'theoretical_availability':'total_available_minutes_in_sim'})

    total_avail_minutes["callsign_group"] = total_avail_minutes["callsign"].apply(lambda x: re.sub('\D', '', x))


    return theoretical_availability_df, total_avail_minutes


In [260]:
# def calculate_theoretical_time_v2(params_df,
# rota_data=pd.read_csv("../actual_data/HEMS_ROTA.csv"),
# service_data=pd.read_csv("../data/service_dates.csv",
# output_by_month=False)
# ):
#     # Convert data into DataFrames
#     warm_up_end = _processing_functions.get_param("warm_up_end_date", params_df)
#     warm_up_end = datetime.strptime(warm_up_end, "%Y-%m-%d %H:%M:%S")

#     sim_end = _processing_functions.get_param("sim_end_date", params_df)
#     sim_end = datetime.strptime(sim_end, "%Y-%m-%d %H:%M:%S")

# #     hems_rota = pd.read_csv(rota_path)
#     # service_dates = pd.read_csv(service_data_path)

#     date_range = pd.date_range(start=warm_up_end.date(),
#                             end=sim_end.date(),
#                             freq='D')
#     daily_df = pd.DataFrame({'date': date_range})

# #     daily_df = pd.DataFrame(daily_data)
#     rota_df = pd.DataFrame(rota_data)
#     service_df = pd.DataFrame(service_data)

#     # Convert date columns to datetime format
#     daily_df['date'] = pd.to_datetime(daily_df['date'])
#     service_df['service_start_date'] = pd.to_datetime(service_df['service_start_date'])
#     service_df['service_end_date'] = pd.to_datetime(service_df['service_end_date'])

#     # Initialize dictionary to store results
#     theoretical_availability = {}

#     # Iterate over each row in the daily dataset
#     for index, row in daily_df.iterrows():
#         current_date = row['date']

#         # Store theoretical available time for each resource
#         day_data = {}

#         for _, rota in rota_df.iterrows():
#             callsign = rota['callsign']

#             # Determine summer or winter schedule
#             is_summer = current_date.month in range(3, 11)
#             start_hour = rota['summer_start'] if is_summer else rota['winter_start']
#             end_hour = rota['summer_end'] if is_summer else rota['winter_end']

#             # Handle cases where the shift ends after midnight
#             if end_hour < start_hour:
#                 daily_available_time = (24 - start_hour) + end_hour
#             else:
#                 daily_available_time = end_hour - start_hour

#             total_available_time = daily_available_time * 60  # Convert to minutes

#             # Adjust for servicing periods
#             service_downtime = 0
#             for _, service in service_df[service_df['resource'] == callsign].iterrows():
#                 if service['service_start_date'] <= current_date <= service['service_end_date']:
#                     service_downtime = daily_available_time * 60
#                     break

#             # Final available time after accounting for servicing
#             day_data[callsign] = total_available_time - service_downtime

#         theoretical_availability[current_date.strftime('%Y-%m-%d')] = day_data

#     theoretical_availability_df = pd.DataFrame(theoretical_availability).T
#     theoretical_availability_df.index.name = "month"
#     theoretical_availability_df = theoretical_availability_df.reset_index()
#     # theoretical_availability_df = theoretical_availability_df.add_prefix("theoretical_availability_")

#     theoretical_availability_df.fillna(0.0)

#     if long_format_df:
#         theoretical_availability_df = (
#             theoretical_availability_df
#             .melt(id_vars="month")
#             .rename(columns={"value":"theoretical_availability", "variable": "callsign"})
#             )

#         theoretical_availability_df['theoretical_availability'] = theoretical_availability_df['theoretical_availability'].astype('float')

#     return theoretical_availability_df


In [261]:
# calculate_theoretical_time_v2(params_df)

In [262]:
theoretical_availability_df, total_avail_minutes = calculate_theoretical_time_v2(params_df, long_format_df=True)
theoretical_availability_df

,month,callsign,theoretical_availability
0,2025-02-28,H70,1140.0
1,2025-03-01,H70,1140.0
2,2025-03-02,H70,1140.0
3,2025-03-03,H70,1140.0
4,2025-03-04,H70,1140.0
...,...,...,...
1825,2026-02-24,CC72,600.0
1826,2026-02-25,CC72,600.0
1827,2026-02-26,CC72,600.0
1828,2026-02-27,CC72,600.0


In [263]:
total_avail_minutes

,callsign,total_available_minutes_in_sim,callsign_group
0,CC70,417240.0,70
1,CC71,249000.0,71
2,CC72,219600.0,72
3,H70,392160.0,70
4,H71,233160.0,71
